### Pair Programming Decision Tree

Hasta ahora hemos ajustado el modelo usando una Regresión Logística, pero como hemos aprendido, podemos usar el Decision Tree en este tipo de problemas. Los objetivos de este pair programming :

1. Ajustad un modelo de Decision Tree a nuestros datos.

2. Calculad las métricas a nuestro nuevo modelo.

3. Comparad las métricas con el modelo hecho hasta ahora. ¿Cuál es mejor?

In [44]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

1. Ajustad un modelo de Decision Tree a nuestros datos.

2. Calculad las métricas a nuestro nuevo modelo.

3. Comparad las métricas con el modelo hecho hasta ahora. ¿Cuál es mejor?

In [45]:
df=pd.read_csv('../datos/diamonds_def.csv', index_col=0)
df.head(3)

,est_carat,est_depth,est_table,est_price,est_lenght_mm,est_width_mm,est_depth_mm,cut_encoded,clarity_encoded,color_encoded
1,-1.268160,-1.597233,1.661056,-0.986397,-1.646799,-1.705394,-1.780396,5,3,2
2,-1.224362,0.042616,3.519383,-0.986075,-1.503737,-1.498268,-1.780396,4,1,2
3,-1.092965,0.534571,0.267310,-0.983820,-1.369617,-1.354180,-1.316852,2,4,2


In [46]:
#Separamos el df de la VR.
x = df.drop("est_carat", axis = 1) #Este es el df sin la VR.
y = df["est_carat"] #VR.

In [47]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [48]:
# creamos el objeto del modelo, al igual que hacíamos en la regresión lineal
arbol = DecisionTreeRegressor(random_state =0)

# ajustamos el modelo
arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=0)

In [49]:
# fig = plt.figure(figsize = (10,6))
# tree.plot_tree(arbol, feature_names = x_train.columns, filled = True)
# plt.show()

In [50]:
max_features = np.sqrt(len(x_train.columns))
max_features

3.0

In [51]:
print(arbol.tree_.max_depth)

35


In [52]:
# hacemos las predicciones sobre los dos set de datos el X_test y el X_train
y_pred_test_dt = arbol.predict(x_test)
y_pred_train_dt = arbol.predict(x_train)

In [53]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [54]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results1 = metricas(y_test, y_train, y_pred_test_dt, y_pred_train_dt, "Decision Tree I")
dt_results1

,MAE,MSE,RMSE,R2,set,modelo
0,2.434035e-02,1.702532e-02,1.304811e-01,0.982897,test,Decision Tree I
1,2.685032e-17,8.413105e-33,9.172298e-17,1.000000,train,Decision Tree I


In [55]:
# lo primero que tenemos que hacer es definir un diccionario con los hiperparámetros que queremos modificar y los valores que queremos 

param = {"max_depth": [3,7,17], #La pofundidad del arbol es de 35. Facilitamos parámetros que nos permitan dividir por la mitad para ajustar lo máximo posible.
        "max_features": [1,2,3],#El resultado de la raiz cuadrada es de 3 por lo que pasamos parametros inferiores para volver a ajustar correctamente.
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split": [500, 1000, 1500], #Asociamos estos tres parametros al nodo madrepara reducir el overfitting y ajustarnos en la medida de lo posible relacion lineal.
        "min_samples_leaf": [300,800,1300]} #Asociamos estos tres parametros al nodo hijo para reducir el overfitting y ajustarnos en la medida de lo posible relacion lineal.

In [56]:
# una vez creado el diccionario iniciaremos el modelo con GridSearch
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [57]:
gs.fit(x_train, y_train)

In [ ]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=17, max_features=3, min_samples_leaf=300,
                      min_samples_split=500)

In [ ]:
# veamos ahora que pinta tiene nuestro árbol
# fig = plt.figure(figsize=(40, 20))
# tree.plot_tree(mejor_modelo, feature_names=x_train.columns, filled=True);

In [ ]:
y_pred_test_dt2 = mejor_modelo.predict(x_test)
y_pred_train_dt2 = mejor_modelo.predict(x_train)

In [ ]:
dt_results2 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "Decision tree II")
dt_results2

,MAE,MSE,RMSE,R2,set,modelo
0,0.066220,0.042396,0.205903,0.957411,test,Decision tree II
1,0.063966,0.040035,0.200088,0.959996,train,Decision tree II


In [ ]:
# vamos  a juntar los dataframes de los resultados de los modelos para poder compararlos mejor

df_decision_results = pd.concat([dt_results1, dt_results2], axis = 0)
df_decision_results

,MAE,MSE,RMSE,R2,set,modelo
0,2.434035e-02,1.702532e-02,1.304811e-01,0.982897,test,Decission Tree I
1,2.685032e-17,8.413105e-33,9.172298e-17,1.000000,train,Decission Tree I
0,6.621966e-02,4.239591e-02,2.059027e-01,0.957411,test,Decision tree II
1,6.396635e-02,4.003527e-02,2.000882e-01,0.959996,train,Decision tree II


In [ ]:
#si recodáis, en la clase de métricas guardamos en un csv los resultados de las métricas del modelo de regresión lineal
# hoy vamos a cargar ese csv y lo vamos a unir al dataframe que acabamos de crear con los resultados del decision tree para comparar las métricas
df_resultados_metricas = pd.read_csv("../datos/diamonds_rdos_metricas.csv", index_col=0)
df_resultados_metricas.head(2)

,MAE,MSE,RMSE,R2,set,modelo
0,0.152142,0.074400,0.272763,0.925262,test,Linear Regresion
1,0.149893,0.071864,0.268074,0.928192,train,LinearRegression


In [ ]:
df_metricas_RL_DT = pd.concat([df_decision_results, df_resultados_metricas ], axis = 0)
df_metricas_RL_DT.head(6)

,MAE,MSE,RMSE,R2,set,modelo
0,2.434035e-02,1.702532e-02,1.304811e-01,0.982897,test,Decission Tree I
1,2.685032e-17,8.413105e-33,9.172298e-17,1.000000,train,Decission Tree I
0,6.621966e-02,4.239591e-02,2.059027e-01,0.957411,test,Decision tree II
1,6.396635e-02,4.003527e-02,2.000882e-01,0.959996,train,Decision tree II
0,1.521418e-01,7.439976e-02,2.727632e-01,0.925262,test,Linear Regresion
1,1.498925e-01,7.186377e-02,2.680742e-01,0.928192,train,LinearRegression


> Vemos que el modelo que mejor se ajusta es el obtenido en el primer arbol de Decision Tree I, donde apreciamos que en el entrenamiento predice de manera perfecta (R2=1), y el resto de las metricas, y en el resto de las métricas, es el modelo que más se acerca al 0. Reflejando, por lo tanto, un mejor ajuste que el resto de modelos.